In [1]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# anime = pd.read_csv("/Users/adithyaabhishek/Desktop/USML/project/report/data/anime.csv")
# anime_desc = pd.read_csv('/Users/adithyaabhishek/Desktop/USML/project/report/data/anime_with_synopsis.csv')
# rating = pd.read_csv('/Users/adithyaabhishek/Desktop/USML/project/report/data/rating_complete.csv')
anime = pd.read_csv("anime.csv")
anime_desc = pd.read_csv('anime_with_synopsis.csv')
rating = pd.read_csv('rating_complete.csv')

In [4]:
anime_merged = pd.merge(anime_desc,anime[['MAL_ID','Type','Popularity','Members','Favorites']],on='MAL_ID')
anime_merged.head(5)

,MAL_ID,Name,Score,Genres,sypnopsis,Type,Popularity,Members,Favorites
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever...",TV,39,1251960,61971
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ...",Movie,518,273145,1174
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0...",TV,201,558913,12944
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...,TV,1467,94683,587
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...,TV,4369,13224,18


In [5]:
anime_merged["Score"].describe()

count       16214
unique        532
top       Unknown
freq         5123
Name: Score, dtype: object

In [6]:
anime_merged = anime_merged[(anime_merged["Score"] != "Unknown")] 
anime_merged.shape

(11091, 9)

In [7]:
anime_merged['sypnopsis'] = anime_merged['sypnopsis'].fillna('')

In [8]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(anime_merged['sypnopsis'])
tfidf_matrix.shape

(11091, 386042)

In [9]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_similarity.shape

(11091, 11091)

In [10]:
anime_merged = anime_merged.reset_index()
titles = anime_merged['Name']
indices = pd.Series(anime_merged.index, index=anime_merged['Name'])

## Content Based

In [11]:

def content_recommendations(title):
    idx = indices[title]
    similarity_scores = list(enumerate(cosine_similarity[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:31]
    anime_indices = [i[0] for i in similarity_scores]
    
    anime_list = anime_merged.iloc[anime_indices][['Name', 'Members', 'Score']]
    favorite_count = anime_list[anime_list['Members'].notnull()]['Members'].astype('int')
    score_avg = anime_list[anime_list['Score'].notnull()]['Score'].astype('float')
    C = score_avg.mean()
    m = favorite_count.quantile(0.60)
    qualified = anime_list[(anime_list['Members'] >= m) & (anime_list['Members'].notnull()) & (anime_list['Score'].notnull())]
    qualified['Members'] = qualified['Members'].astype('int')
    qualified['Score'] = qualified['Score'].astype('float')
    def weighted_rating(x):
        v = x['Members']
        R = x['Score']
        return (v/(v+m) * R) + (m/(m+v) * C)   
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    
    return qualified


In [38]:
content_recommendations('Naruto').head(10)

,Name,Members,Score,wr
1506,Naruto: Shippuuden,1543765,8.16,8.051279
824,Higurashi no Naku Koro ni,638491,7.95,7.762043
5998,The Last: Naruto the Movie,352160,7.76,7.528764
5623,Naruto: Shippuuden Movie 6 - Road to Ninja,223826,7.67,7.400756
7245,Boruto: Naruto the Movie,320603,7.50,7.342222
4381,Naruto: Shippuuden Movie 4 - The Lost Tower,172051,7.42,7.231966
2089,Naruto: Shippuuden Movie 1,211544,7.29,7.178705
3145,Naruto: Shippuuden Movie 2 - Kizuna,188680,7.29,7.171518
403,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,215046,7.10,7.072324
826,Naruto Movie 2: Dai Gekitotsu! Maboroshi no Ch...,172509,6.88,6.956515


In [15]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 5.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626486 sha256=703b3733d7dde62699fb2dc9babbb89fa2ea49bd438335ff8c75c01c1431d2af
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [19]:
#Collaborative
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

import warnings; warnings.simplefilter('ignore')

rating.head(10)

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9
5,0,431,8
6,0,578,10
7,0,433,6
8,0,1571,10
9,0,121,9


In [20]:
rating['rating'].value_counts()

8     14642156
7     13325549
9      9773857
6      6849293
10     6716048
5      3436250
4      1455102
3       696048
2       405556
1       333419
Name: rating, dtype: int64

In [21]:
rating = rating[(rating["rating"] != -1)]

In [22]:
rating.head(5)

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9


In [23]:
reader = Reader()
rating_data = Dataset.load_from_df(rating, reader)
svd = SVD()

In [24]:
trainset = rating_data.build_full_trainset()

In [25]:
svd.fit(trainset)

In [26]:
svd.predict(1, 356, 5)

Prediction(uid=1, iid=356, r_ui=5, est=5, details={'was_impossible': False})

In [27]:
id_map = anime_merged[['MAL_ID']]
id_map['id'] = list(range(1,anime_merged.shape[0]+1,1))
id_map = id_map.merge(anime_merged[['MAL_ID', 'Name']], on='MAL_ID').set_index('Name')

In [28]:
indices_map = id_map.set_index('id')

In [29]:
def hybrid_recommendations(user_id,title):
    idx = indices[title]
    similarity_scores = list(enumerate(cosine_similarity[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:31]    
    anime_indices = [i[0] for i in similarity_scores]
            
    anime_list = anime_merged.iloc[anime_indices][['MAL_ID','Name', 'Members', 'Score','Genres']]
    favorite_count = anime_list[anime_list['Members'].notnull()]['Members'].astype('int')
    score_avg = anime_list[anime_list['Score'].notnull()]['Score'].astype('float')
    C = score_avg.mean()
    m = favorite_count.quantile(0.60)
    qualified = anime_list[(anime_list['Members'] >= m) & (anime_list['Members'].notnull()) & (anime_list['Score'].notnull())]    
    qualified['Members'] = qualified['Members'].astype('int')
    qualified['Score'] = qualified['Score'].astype('float')
    def weighted_rating(x):
        v = x['Members']
        R = x['Score']
        return (v/(v+m) * R) + (m/(m+v) * C)   
     
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(30)    
    
    qualified['id'] = list(range(1,qualified.shape[0]+1,1))  
    qualified['est'] = qualified['id'].apply(lambda x: svd.predict(user_id, indices_map.loc[x]['MAL_ID']).est)
    qualified = qualified.sort_values('est', ascending=False)
    res = qualified[['MAL_ID','Name','Genres','Score']]
    return res.head(10)    


In [37]:
hybrid_recommendations(100,'Naruto')

,MAL_ID,Name,Genres,Score
1506,1735,Naruto: Shippuuden,"Action, Adventure, Comedy, Super Power, Martia...",8.16
824,934,Higurashi no Naku Koro ni,"Mystery, Dementia, Horror, Psychological, Supe...",7.95
5998,16870,The Last: Naruto the Movie,"Action, Super Power, Romance, Martial Arts, Sh...",7.76
5623,13667,Naruto: Shippuuden Movie 6 - Road to Ninja,"Action, Adventure, Super Power, Martial Arts, ...",7.67
7245,28755,Boruto: Naruto the Movie,"Action, Comedy, Martial Arts, Shounen, Super P...",7.50
4381,8246,Naruto: Shippuuden Movie 4 - The Lost Tower,"Action, Comedy, Martial Arts, Shounen, Super P...",7.42
2089,2472,Naruto: Shippuuden Movie 1,"Action, Adventure, Comedy, Fantasy, Shounen",7.29
3145,4437,Naruto: Shippuuden Movie 2 - Kizuna,"Action, Martial Arts, Shounen, Supernatural",7.29
403,442,Naruto Movie 1: Dai Katsugeki!! Yuki Hime Shin...,"Adventure, Comedy, Historical, Supernatural, D...",7.10
826,936,Naruto Movie 2: Dai Gekitotsu! Maboroshi no Ch...,"Adventure, Comedy, Drama, Fantasy, Shounen, Su...",6.88
